## 1. 필요한 라이브러리 불러오기

In [53]:
from pathlib import Path
import pandas as pd
import sys

ROOT_DIR = Path.cwd().parent.parent
sys.path.append(ROOT_DIR)


## 2. 데이터 준비하기


### 2.1 원본 데이터 읽어오기

In [55]:
df1 = pd.read_json(
    ROOT_DIR / "datasets/민원(콜센터) 질의응답_다산콜센터_대중교통 안내_Training.json"
)
df2 = pd.read_json(
    ROOT_DIR / "datasets/민원(콜센터) 질의응답_다산콜센터_대중교통 안내_Validation.json"
)
df1.shape, df2.shape

((38966, 15), (4863, 15))

### 2.2 데이터프레임 합치기

In [56]:
# 전체 데이터셋중 개체명에 "도시철도, 지하철" 이 있는 경우만 추출함.
merged = pd.concat([df1, df2], ignore_index=True)
merged.shape

(43829, 15)

### 2.3도시철도 관련 데이터만 추출하기

In [57]:
merged["용어사전"].value_counts()
keywords = ["자하철", "도시철도", "호선"]

# '용어사전' 컬럼에 키워드가 하나라도 포함된 행만 추출
subway_df = merged[
    merged["용어사전"].apply(lambda x: any(kw in str(x) for kw in keywords))
]

print(subway_df.shape)

(471, 15)


### 2.4 도시철도 관련 질문 모두 수집하기

In [59]:
customer_questions = list(subway_df["고객질문(요청)"].unique())
for q in customer_questions:
    print(q)
    print("-" * 100)
print(len(customer_questions))


1호선 첫 차가 몇시 운영이에요?
----------------------------------------------------------------------------------------------------

----------------------------------------------------------------------------------------------------
지하철도 통제가 되나요?
----------------------------------------------------------------------------------------------------
그럼 9호선은 아예 자전거 가지고 타는 게 불가능한가요?
----------------------------------------------------------------------------------------------------
몇호선 타야해요?
----------------------------------------------------------------------------------------------------
고속터미널은 3호선 아니었나요?
----------------------------------------------------------------------------------------------------
1호선 급행도 노량진에서 하차하나요?
----------------------------------------------------------------------------------------------------
9호선도 급행이 있나요?
----------------------------------------------------------------------------------------------------
9호선은 얼마나 타야 종합운동장 나오나요?
---------------------------------------

### 2.5 LLM으로 질문 전처리하기
  - 컨텍스트 길이를 고려해서 절반씩 나눠서 LLM 호출
  - 단순한 요약 및 재작성 태스크이므로 모델은 비교적 작은 gpt-4o-mini 사용
  - 답변은 json 형식으로 반환

In [74]:
import openai


def preprocess_question(question):
    messages = [
        {
            "role": "system",
            "content": "당신은 대중교통 관련 질문을 전처리하는 도우미입니다.",
        },
        {
            "role": "user",
            "content": f"""다음은 대중교통 관련 민원인의 질문 85개입니다.
            이 질문들을 참고해서 다음의 작업을 수행하세요
            1. 중복된 질문은 하나로 줄이고, 중복되지 않는 질문은 그대로 두세요.
            2. 질문을 정확하고 구체적으로 다시 작성하세요.
            3. 일반적인 대중교통 질문이 아닌 경우 제외하세요.
            4. 이외에도 맥락상 필요하다고 생각하는 질문이 있는 경우 추가하세요.
            5. 최종적으로 전처리된 질문을 반환하세요.
            답변은 json 형식으로 반환하세요.
            ---
            질문:
            {question}
            """,
        },
    ]

    try:
        response = openai.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            response_format={"type": "json_object"},
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"API 호출 중 오류 발생: {e}")
        # 응답 형식 옵션을 제거하고 재시도
        try:
            response = openai.chat.completions.create(
                model="gpt-4o-mini", messages=messages
            )
            return response.choices[0].message.content.strip()
        except Exception as e:
            print(f"두 번째 시도 중 오류 발생: {e}")
            return None


q1 = preprocess_question("\n".join(customer_questions[:40]))
q2 = preprocess_question("\n".join(customer_questions[40:]))


### 2.6 전처리 결과 합치고 평가하기
  - 전처리 결과 현재의 RAG 데이터로 답변할 수 없는 질문이 대부분(길찾기 등)
  - 이 질문 세트는 제외하기로 결정함

In [76]:
import json

q1_dict = json.loads(q1)
q2_dict = json.loads(q2)

questions = q1_dict["questions"] + q2_dict["questions"]
print(len(questions))
questions


64


[{'question': '1호선의 첫 차 운영 시간은 몇 시인가요?'},
 {'question': '지하철도 통제가 가능한가요?'},
 {'question': '9호선에서 자전거를 가지고 타는 것이 불가능한가요?'},
 {'question': '강남으로 가기 위해서 어떤 호선을 이용해야 하나요?'},
 {'question': '고속터미널역은 3호선에 위치한 게 맞나요?'},
 {'question': '1호선 급행열차는 노량진역에서 하차하나요?'},
 {'question': '9호선에도 급행열차가 존재하나요?'},
 {'question': '9호선에서 종합운동장역까지는 얼마나 걸리나요?'},
 {'question': '오산시청역에서 강남으로 가는 버스는 몇 번인가요?'},
 {'question': '시청에서 강남까지 이동하는 데 소요되는 시간은 얼마나 되나요?'},
 {'question': '신풍역에서 동작역까지 가는 경로를 알려줄 수 있나요?'},
 {'question': '세종문화회관이 종점인가요?'},
 {'question': '동대문역사는 2호선으로 가나요?'},
 {'question': '4호선에서 가장 많은 승객이 있는 역은 어디인가요?'},
 {'question': '4호선에는 승객이 많은 급행역이 있나요?'},
 {'question': '2호선과 신분당선의 막차 시간은 연결되어 있나요?'},
 {'question': '수서역에서 삼성병원으로 가는 버스는 있나요? 아니면 지하철을 이용해야 하나요?'},
 {'question': '4호선은 어떤 방향으로 승차해야 하나요?'},
 {'question': '롯데월드를 이용할 때 지하철 할인 요금이 있나요?'},
 {'question': '충무로역은 몇 호선에 위치해 있나요?'},
 {'question': '대전에는 지하철 노선이 없나요?'},
 {'question': '환승할 때 1호선에서 2호선으로 바꿔 타야 하나요?'},
 {'question': '선정릉역과 등촌역이 둘 다 9호선인가요?'},
 {'

## 3. 도큐먼트 데이터 전처리 및 적재

1. 부산교통공사 홈페이지의 "이용안내" 섹션

    <img src="/Users/sdh/Dev/projects/humetro-ai-assistant/assets/부산교통공사_이용안내.png" width="50%" alt="대체텍스트">


2. 부산교통공사 직원 교육용 교재
 - "일타역무 교재"
3. 부산교통공사 역무정보 편집본
 - "역무 위키" 

### 3.1 마크다운 형식으로 저장된 데이터 전처리(with llm)

In [81]:
PROMPT = f"""
아래의 웹에서 수집한 정보를 
"""


def preprocess_markdown(markdown_text: str) -> str:
    """
    마크다운 형식의 데이터를 전처리하는 함수
    """
    openai.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": markdown_text},
        ],
    )


rag_docs_dir = ROOT_DIR / "datasets/Rag_Docs"
md_files = list(rag_docs_dir.glob("*.md"))
print(md_files)
print(len(md_files))


for md_file in md_files:
    with open(md_file, "r") as f:
        markdown_text = f.read()
    preprocessed_text = preprocess_markdown(markdown_text)
    print(preprocessed_text)


[PosixPath('/Users/sdh/Dev/projects/humetro-ai-assistant/datasets/Rag_Docs/에스컬레이터 목록  교통약자 배려시설  이용안내  부산교통공사.md'), PosixPath('/Users/sdh/Dev/projects/humetro-ai-assistant/datasets/Rag_Docs/만남의 장소 목록  편의시설  이용안내  부산교통공사_list02.do?gubun=1&menu_no=1001010801.md'), PosixPath('/Users/sdh/Dev/projects/humetro-ai-assistant/datasets/Rag_Docs/긴급상황 대피요령  긴급상황 대피요령  도시철도 이용  이용안내  부산교통공사.md'), PosixPath('/Users/sdh/Dev/projects/humetro-ai-assistant/datasets/Rag_Docs/전동휠체어 급속충전기 목록  교통약자 배려시설  이용안내  부산교통공사.md'), PosixPath('/Users/sdh/Dev/projects/humetro-ai-assistant/datasets/Rag_Docs/엘리베이터 목록  교통약자 배려시설  이용안내  부산교통공사_list.do?gubun=2&menu_no=1001010702.md'), PosixPath('/Users/sdh/Dev/projects/humetro-ai-assistant/datasets/Rag_Docs/엘리베이터 목록  교통약자 배려시설  이용안내  부산교통공사.md'), PosixPath('/Users/sdh/Dev/projects/humetro-ai-assistant/datasets/Rag_Docs/부산교통공사.md'), PosixPath('/Users/sdh/Dev/projects/humetro-ai-assistant/datasets/Rag_Docs/연결 통로  편의시설  이용안내  부산교통공사.md'), PosixPath('/Users/sdh/Dev/projects/hu

## 4. 도큐먼트를 바탕으로 Q/A 데이터 합성

## 5. 